In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [2]:
base_url = 'https://www.slmpd.org/'
page_url = 'https://www.slmpd.org/crime_stats.shtml'

In [9]:
# df = pd.DataFrame(data={'LastUpdated': ['January 2024']})
# date_string = df.iloc[0,0]

In [10]:
# with sqlite3.connect('data/database.db') as conn:
#     df = pd.read_sql_query("SELECT LastUpdated FROM meta_data", conn)
    
# date_string = df.iloc[0,0]
date_object = parser.parse(date_string)
next_month = date_object + relativedelta(months=1)
next_month_string = next_month.strftime("%B%Y")
next_month_string

'February2024'

In [7]:
page = requests.get(page_url)
soup = BeautifulSoup(page.content, "html.parser")
block = soup.find('div', attrs = {'class': 'news_div', 'id': 'news_div'})

entries = block.findAll('a')
recent_entry = entries[-1]
tail_url = recent_entry['href']

if next_month_string in tail_url:
    csv_link = base_url + tail_url
    response = requests.get(csv_link)

'NIBRSData/April2024.csv'

In [11]:
next_month_string in tail_url

False

In [ ]:
recent_entry

In [ ]:
csv_link = base_url + tail_url
response = requests.get(csv_link)

In [ ]:
csv_filename = next_month_string + '.csv

with open('April2024.csv', 'wb') as f:
    f.write(response.content)

print("CSV file downloaded successfully.")

In [ ]:
j = datetime.strptime(f'{curr_entry_month} {curr_entry_year}', '%B %Y')
j

In [ ]:
j.strftime()